**Связь docker контейнера и файловой системы хоста**

# Команда `cp`

Поможет провести коппирование в работающий контейнер. Синтаксис использования комманды следующий:

`docker cp <путь на хосте> <контейнер в который производится копирование>:<путь в контейнере>`

Далее пример, как я в работающий контейнер закину файл "my_datafile".

In [ ]:
%%bash
# поднимаю контейнер в фоновом режиме
docker run -itd --rm --name c1 file_example
echo ""
# показываю что в конейнере нету файла
docker exec c1 ls
echo ""
# копирую файл через cp
docker cp my_datafile.csv c1:experimental/my_datafile.csv
# сомтрю в папку теперь
docker exec c1 ls
docker stop c1

0a4764eb41d1a3675983044244de0a3d4f891309f3c436413db479839b21abb5

experimental.py

experimental.py
my_datafile.csv
c1


# Bind mount

Можно примонтировать к контейнеру любую папку или файл в любом месте на хосте. **Основное предназначение** - входная информация по типу ключей.

Bind mount создается через использование опции `--v <полный путь к файлу/папке на хосте:полный путь к файлу папке в контейнере>` комманды `run`.

In [2]:
%%bash
# убежадемся, что изначально файл пустой
cat my_datafile.csv
# потом запускаем контейнер монтируя bindmount
docker run --rm -itd -v $(pwd)/my_datafile.csv:/experimental/my_datafile.csv --name bm_ex file_example
# запускаем программу, которая должна добавить строчку в файл
docker exec bm_ex python3 experimental.py
# проверяем файл НА ХОСТЕ
cat my_datafile.csv

docker stop bm_ex

col1,col2
9f09b068890c28544b1892a4ba050e128016703c847248ba0d96a0f8861ba506
col1,col2
0.48907447800000703,0.3088149216473689
bm_ex


### Тонкости


##### root контейнера

Даже если для хоста файл или папка имеют особый уровень доступа, **контейнер всегда работает под root**. Таким образом если монтировать такую папку, это может привести к тому, что в ней начнутся несанкционированные изменения. Далее пример.

In [3]:
%%bash
# создаем папку и в ней файл и даже в него записываем
# сверхсекретное сообщение
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# закрываю доступ в папку
chmod 000 secret_dir
# убедимся, что через хост мы не может не получить инфу
# ни удалить её
cat secret_dir/secret_file
rm secret_dir/secret_file
# поднимаем контейнер и монтируем в него данную папку
docker run --rm -itd -v $(pwd)/secret_dir:/experimental/secret_dir --name perm_ex file_example
# и вауля, легко достаем "секретную информацию"
docker exec perm_ex cat secret_dir/secret_file
# или даже можем удалить файл
docker exec perm_ex rm secret_dir/secret_file

docker stop perm_ex

cat: secret_dir/secret_file: Permission denied
rm: cannot remove 'secret_dir/secret_file': Permission denied


d02a3365f075b0dcde14a68698f77b8e1ad89eb8ad1d112848c2a3dd037f37b1
super secret info
perm_ex


##### Опция `ro`

Продолжая предыдущий подраздел, заметим, что при монтировании чего либо можно указать опцию `ro` (read only), которая запретит контейнеру изменять примонтированный файл. Но по какой-то причине, это работет не во всех случаях, так следующий пример ломает систему:

In [5]:
%%bash
echo "some data" > ro_ex
# поднимаю контейнер с опцией ro
docker run --rm -idt -v $(pwd)/ro_ex:/experimental/ro_ex:ro --name ro_ex file_example
# изменяю файл
docker exec ro_ex echo "new some data" > ro_ex
# вывожу новый файл
cat ro_ex
docker stop ro_ex

22a5ea7e72f7fb51efbecaed558ae4cf4f11cd5acebacb8cec45f11223937c44
new some data
ro_ex


Тем не менне если воти в контейнер и пытаться изменнить например через nano - то дейсвительно убдет ошибка.

##### Вход с указанием пользователя

Так же проблему с тем что в контейнере постоянно root можно обойти, указав пользователя при запуске контейнера (опция `-u=<id>`). Так пример из подраздела `root контейнера` можно сделать следующим образом:

In [6]:
%%bash
# создаем папку и в ней файл и даже в него записываем
# сверхсекретное сообщение
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# закрываю доступ в папку
chmod 000 secret_dir

# поднимаем контейнер и монтируем в него данную папку
docker run --rm -itd -v $(pwd)/secret_dir:/experimental/secret_dir -u=1000 --name perm_ex file_example
# и теперь то эта опция вызовет предупреждение "Permission denied"
docker exec perm_ex cat secret_dir/secret_file

docker stop perm_ex

552f8215a63651c234f63ee48d7948b34890c4a66faa5c89dbc9ede8fe0830fc


cat: secret_dir/secret_file: Permission denied


perm_ex


# Volume

Это файлы которые лежат на хосте и не зависят от конкретного образа, но зависят от docker в целом. **Основное предназначение** - хранение данных.

### Комманды `volume`

Любая комманда для работы с volume начитается с `docker volume`.

- `ls` - отобразить доступные volumes;
- `create` - создать volume;
- `rm` - удалить volume;
- `prune` - удалить valumes которые не используются в каком-либо контейнерe;
- `inspect` - позволяет получить информацию о volume (в том числе где он лежит на хосте).

### Пример использования volume

Пример разложился в несколько ячеек, потому следующие ячейки следует запускать в строгой последовательносяти.

Убедимся, что изначально файл "my_datafile.csv" пустой.

In [2]:
%%bash
cat my_datafile.csv

col1,col2


Созданный ранее volume передается контейнеру через опцию `-v <имя volume>:<директория в контейнере>`

In [4]:
%%bash
# создание volume
docker volume create ex_volume
# запускаем контейнер и прокидываем в него наш volume
docker run --rm -itd -v ex_volume:/experimental --name volume_ex file_example

ex_volume
56e1e4a3dbc4f83a8f6a5eae3185064d323cfdcf02a3142485b6af84b08eb6c5


И так после исполнения предыдущей ячейки у нас папка "experimental" контейнера будет связана с "ex_volume". Сейчат, например, через `cp` добавим закинем в volume наш файл. И создадим в нем строчку. И удалим остановим контейнер.

In [6]:
%%bash
# копирование файла с информацией
docker cp my_datafile.csv volume_ex:/experimental/my_datafile.csv
# запуск программы, которая досоздаст информацию
docker exec volume_ex python3 experimental.py
# остановка контейнера
docker stop volume_ex

volume_ex


Теперь по логике, то, что насоздавалось за время работы контейнера можно достать из volume. Только нужно узнать где именно он лежит:

In [7]:
%%bash
docker volume inspect ex_volume

[
    {
        "CreatedAt": "2023-02-22T19:28:29+03:00",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/ex_volume/_data",
        "Name": "ex_volume",
        "Options": null,
        "Scope": "local"
    }
]


Тут "Mountpoint" и указыват на ту папку в которой был создан volume. Для того, чтобы зайти в него нужен root. потому привожу результаты скрином.

<img src="inside_volume.png"></img>

И, что самое крутое, на месте этого образа, я могу поднять точно такойже или совсем другой, и прицепить к нему все теже данные, которые были использованы в прошлый раз. Так в следующей ячейке я поднимаю, точно такой-же образ, что и раньше, заглянув в "my_datafile.csv" нахожу все тот-же файл.

In [13]:
%%bash
# поднимаю контейнер
docker run --rm -itd -v ex_volume:/experimental --name volume_ex file_example
# заглядываю в изучаемый файл
docker exec volume_ex cat my_datafile.csv
# останавилваю контейнер
docker stop volume_ex

a94478c4cbd5319c8cc25000207c3a0a8a1ac00ee338ef8cdf51821581e79421
col1,col2
0.13246072579884416,0.28795926573494846
volume_ex


### Тонкости

Некоторые контейнеры, когда поднимаются создают себе volume автоматически. Потому в один момент можно обнаружить, что у тебя весь диск забит. Например `yandex/clickhouse-server`. Так в следующем примере я поднимаю несколько контейнеров `yandex/clickhouse-server` и покзываю, что каждый из них создал по volume.

In [ ]:
%%bash
# поднимаю clickhouse
docker run -d --name db_1 --rm yandex/clickhouse-server
docker run -d --name db_2 --rm yandex/clickhouse-server
docker run -d --name db_3 --rm yandex/clickhouse-server

# смотрю список volume -
docker volume ls
docker stop db_1 db_2 db_3

82d50d878c1babea30805e20f9aaedf0198fd2617207f2891528f190d210bbad
36ae71fb6eb78cc91e958039ab53f093807faa43c82915af6a76b4ef482da0ce
60dc2f71e2b29277b8eda6be533ab42547d0061b64f115b87a1052a1541bb7c9
DRIVER    VOLUME NAME
local     2a6676858530a9d28db91bcea872748ad2516ff712716293c3cd42864f79669d
local     994071ad6640596efb04194a5c396696269a1c7bbd6c3d61861899f5bac64396
local     ef1d5dfad191f8c46fff3c140b26b40d3e89f2adefbb66de44c17d9e4d36e5e3
db_1
db_2
db_3
